# Imports

In [56]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm

from sympy import Symbol, solve # 방정식 해 구하기

# Data settings

## Load dataset
* 데이터셋 불러오기
* `net` : 네트워크 파일
* `OD` : Origin-Destination 수요

In [57]:
data_dir = 'dataset'

In [58]:
network_file = 'ex1_Network.csv'
network_path = os.path.join(data_dir, network_file)

In [59]:
OD_file = 'ex1_OD.csv'
OD_path = os.path.join(data_dir, OD_file)

In [60]:
network = pd.read_csv(network_path, encoding = 'UTF-8')

In [61]:
OD = pd.read_csv(OD_path, encoding = 'UTF-8')

In [62]:
network.head(3)
#> Init node

,Init node,Term node,Free Flow Time,Power,Capacity,B,Length
0,1,2,1,4,1,0.15,1
1,1,3,1,4,3,0.15,1
2,3,2,1,4,2,0.15,1


In [63]:
OD.tail(20).T

,0,1
O,1,3
D,2,2
T,2,2


## Preprocessing
* 필요 변수
    * `O` : 기점
    * `D` : 종점
    * `Link_num` : 링크번호
    * `FFT` : Free Flow Time
    * `Alpha`, `Beta` : BPR 함수에 필요한 상수들
    * `Length` : 네트워크 길이
    * `Toll` : 네트워크 요금
    * `Volume` : 링크 통행량
* `BPR Function` : `Cost = Free Flow Time * (1 + (Alpha * (Xn / Capacity))**Beta) + Length * Toll`

In [64]:
def preprocessing(network):
    """네트워크 객체의 컬럼 이름 및 순서를 정리해줌
    - O, D, FFT, alpha, Capacity, beta, Length 등 만 남기고 나머지는 지운다.
    """
    net = pd.DataFrame({
        'O' : network['Init node '],
        'D' : network['Term node '],
        'FFT' : network['Free Flow Time '],
        'Power' : network['Power'],
        'Capacity' : network['Capacity '],
        'B' : network['B'].round(2),
        'Length' : network['Length '],
        #'Toll' : network['Toll '],
        'Link_num' : [i for i in range(1, len(network)+1)],
        'x_n' : 0,
        'y_n' : 0,
        'Direction' : 0,
        'x_n1' : 0,
        'Cost' : 0
    })
   
    net = net.sort_values(by = ['O', 'D']) # net1 데이터프레임을 O, D에 대하여 오름차순 정렬
    net.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return net

In [65]:
# net_cost.info()
# 데이터 축소 :: int64 -> int16, float64 -> float16으로 바꾸어 데이터 크기를 단축해 준다.

def type_change(network):
    """net_cost네트워크의 변수들 타입을 int16, float16으로 변화시켜 데이터 메모리 절감"""
    network[['O', 'D', 'FFT', 'Power', 'Length', 'Link_num']] = network[['O', 'D', 'FFT', 'Power', 'Length', 'Link_num']].astype(np.int16)
    network[['Capacity', 'B', 'x_n', 'y_n', 'Direction', 'x_n1', 'Cost']] = network[['Capacity', 'B', 'x_n', 'y_n', 'Direction', 'x_n1', 'Cost']].astype(np.float16)

    return network

# Shortest Path Definition
* `Dijkstra` 방식을 사용하여 각 O-D별 최단경로를 추출하는 함수 생성

## Node Range

In [66]:
node_range(net1)

,origin,start_link,end_link
origin,,,
1,1,1,2
3,3,3,4
4,4,5,5


In [67]:
def node_range(network):
    """Sioux-Falls Network의 각 기점(Origin)에서 출발하는 링크 번호의 범위를 데이터프레임에 저장하는 함수"""
    origin_nodes = network['O'].unique()
    start_list = []
    end_list = []
    
    for origin in origin_nodes:
        link_list = network[network['O'] == origin]['Link_num'] # 기점이 origin인 모든 링크 번호 리스트 추출
        link_list.sort_values() # 기점 origin에 연결된 링크 번호들을 오름차순으로 정렬
        
        start = link_list.iloc[0] # 링크 리스트의 첫번째 링크(가장 먼저 번호 링크)
        end = link_list.iloc[-1] # 링크 리스트의 마지막 링크(가장 마지막 번호 링크)
        
        start_list.append(start)
        end_list.append(end)
        
    node_link_df = pd.DataFrame({
        'origin' : origin_nodes,
        'start_link' : start_list,
        'end_link' : end_list
    })
    
    node_link_df.set_index('origin', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return node_link_df

## Calculate Cost : $t_a(x_a)$
* 매 iteration마다 갱신되는 $\alpha$, $x_{n}$ 값을 이용해서 Cost를 계산해 주는 함수
* 따라서 iteration을 돌 때마다 다시 계산해주어야 한다.

In [68]:
def calculate_initial_cost(network):
    """
    BPR 함수를 사용하여 링크 통행비용을 계산하는 함수.
    최초 비용을 계산해주므로, Cost 자리에 Free Flow Time을 적용해야 한다.
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost = Free Flow Time * (1 + B * (X_n / Capacity)**Power)
    """
    network['Cost'] = network['FFT'] * (1 + network['B'] * (network['x_n'] / network['Capacity'])**network['Power']) #+ (net1['Length'] * net1['Toll'])
    
    return network

In [69]:
def calculate_iterative_cost(network):
    """
    BPR 함수를 사용하여 Iteration을 돌면서 링크 통행비용을 계산하는 함수. 
    매 iteration마다 갱신되는 x_n 등을 이용하므로, 매 iteration마다 다시 계산해주어야 한다.
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost1 = Cost * (1 + B * (X_n / Capacity)**Power)
    """
    network['Cost'] = network['Cost'] * (1 + network['B'] * (network['x_n'] / network['Capacity'])**network['Power']) #+ (net1['Length'] * net1['Toll'])
    
    return network

## Shortest Path Algorithm : Dijkstra

In [22]:
def tree_Dijkstra(origin, network):
    """Dijkstra 방법을 이용해 Shortest Path를 구하기 위한 경로 트리를 구해주는 과정.
    이때 network는 반드시 cost 컬럼을 포함하고 있어야 합니다."""
    
    SP_df = pd.DataFrame({
        'To' : network['O'].unique(),
        'From' : 0,
        'C1' : 999,
        'Fin' : 0
    })
    
    SP_df.loc[SP_df['To'] == origin, 'C1'] = 0 # 'To' 열 값이 origin과 같은 행의 C1 열 값을 0으로 치환하기
    SP_df = SP_df.sort_values(by = ['C1', 'To']).reset_index(drop = True) # cost_df 를 C1과 To값에 따라서 정렬하기
    
    node_link = node_range(network) # 각 origin을 기점으로 하는 링크번호 범위 데이터프레임
    
    while True:
        if sum(SP_df['Fin']) == len(SP_df):
            break

        a = SP_df.loc[0, 'To'] # cost_df의 1행 1열 데이터
        ac = SP_df.loc[0, 'C1'] # cost_df의 1행 3열 데이터
        
        SP_df.loc[0, 'Fin'] = 1
        SP_df = SP_df.sort_values(by = ['To']).reset_index(drop = True)
            
        # DN : Dijkstra next updating node, D : Dijkstra iteration term
        node_a = node_link[node_link['origin'] == a] # node_range의 링크 중, origin이 a인 행만 추출
        start_link_a = node_a['start_link'].iloc[0] # origin a에 연결된 link들 중 첫번째 번호
        end_link_a = node_a['end_link'].iloc[0] # origin a에 연결된 link들 중 마지막 번호
            
        DN = network[(network['Link_num'] >= start_link_a) & (network['Link_num'] <= end_link_a)]
        
        DN = DN[['D', 'Cost']]
        DN.rename(columns = {'D' : 'To', 'Cost' : 'C'}, inplace = True)
            
        SP_df = pd.merge(SP_df, DN, on = ['To'], how = 'left') 
            
        SP_df = SP_df.fillna(999) # SP_df의 C 값 중 값이 없어 NaN인 값들을 999로 채우기
        #SP_df = SP_df.sort_values(by = 'To').reset_index(drop = True)
            
        upset = list(SP_df[SP_df['C1'] > (SP_df['C'] + ac)]['To']) ######### 진짜 여기서 죽는줄알았음 헛갈림 주의
        
        #print(upset)
        #print(Pk)
            
        if len(upset) > 0:
            SP_df.loc[SP_df['To'].isin(upset), 'Fin'] = 0
            SP_df.loc[SP_df['To'].isin(upset), 'C1'] = SP_df.loc[SP_df['To'].isin(upset), 'C'] + ac
            SP_df.loc[SP_df['To'].isin(upset), 'From'] = a
            
        SP_df = SP_df.iloc[:, 0:-1]
        SP_df = SP_df.sort_values(by = ['Fin', 'C1', 'To']).reset_index(drop = True)
    
    SP_output = pd.DataFrame({
        'O' : SP_df['From'],
        'D' : SP_df['To'],
        'SP' : SP_df['C1']
    })
    
    SP_output = SP_output.sort_values(by = 'SP').reset_index(drop = True)
    
    return SP_output

In [28]:
def SPD_Dijkstra(origin, destination, network):
    """주어진 네트워크 및 비용(SP)을 바탕으로,
    origin >> destination으로 향하는 최단 경로를 구해주는 함수.
    network 데이터프레임은 반드시 cost 열을 포함하고 있어야 합니다."""
    
    mintree = tree_Dijkstra(origin, network)
    OD_SP = mintree[mintree['D'] == destination]
    
    while True:
        if OD_SP['O'].iloc[0] == origin:
            break
        
        added = mintree[mintree['D'] == OD_SP['O'].iloc[0]]
        OD_SP = pd.concat([added, OD_SP], ignore_index = True)
        
    # 각 O-D에 Node Link 붙이기 : network의 Link_num 이용
    OD_SP = pd.merge(OD_SP, network, on = ['O', 'D'], how = 'left')
    OD_SP = OD_SP[['O', 'D', 'SP', 'Link_num']]
    OD_SP.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
        
    return OD_SP

# Initialization
* **`N = 0`**
* 초기 비용 계산 : $x_0$ = 0 (모든 링크의 통행량 = 0)인 상태에서 비용 계산
* 초기 배정 : All-or-Nothing 방식으로 통행 배정, $x_1$ 획득

## Data Ready

In [84]:
net1 = preprocessing(network)

In [85]:
net1.head(3)

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,1,4,1,0.15,1,1,0,0,0,0,0
2,1,3,1,4,3,0.15,1,2,0,0,0,0,0
3,3,2,1,4,2,0.15,1,3,0,0,0,0,0


## 초기 비용 계산 : $t_a(x_a^0)$
* $x_0$ = 0 (전부 0)임을 계산하여 Cost 계산

In [86]:
net1 = calculate_initial_cost(net1) # x0 

In [87]:
net1.head()

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,1,4,1,0.15,1,1,0,0,0,0,1.0
2,1,3,1,4,3,0.15,1,2,0,0,0,0,1.0
3,3,2,1,4,2,0.15,1,3,0,0,0,0,1.0
4,3,4,1,4,4,0.15,1,4,0,0,0,0,1.0
5,4,2,1,4,5,0.15,1,5,0,0,0,0,1.0


## $x_1$ 획득 : AON Assignment

In [92]:
# 각 OD별로 Dijkstra를 사용해 최단경로를 산출, 
# OD 테이블의 교통량(T)을 각 최단경로에 포함된 링크에 All-or-Nothing으로 배정하기

origin_list = OD['O'].unique()

for origin in tqdm(origin_list): # 각 Origin에 대하여

    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D']
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination:
            SPD = SPD_Dijkstra(origin, destination, net1)
            link_list = list(SPD['Link_num'])
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정
            print(f'(O, D) = ({origin}, {destination}), Path Link_num list = {link_list}, Add_volume = {add_volume}')
            net1.loc[net1['Link_num'].isin(link_list), 'x_n'] += add_volume # All or Nothing 배정
        else:
            pass

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

[1 3]
-------- origin = 1 --------
0    2
Name: D, dtype: int64


IndexError: single positional indexer is out-of-bounds

In [ ]:
net1.head()

In [ ]:
net_cost = net1 #### net 1 보존. Iteration 잘못되어도 여기서 다시 시작하자.

# Iteration

## $t_1$ 계산 : 통행비용 갱신
* Update Link Travel Time
* 전 단계에서 계산한 통행량 $x_1$로 통행비용(Cost) 갱신

In [ ]:
net_cost = calculate_iterative_cost(net_cost) # Cost 갱신

In [ ]:
net_cost.head(20)

## $y_1$ 획득 : AON Assignment
* 전 단계의 $t_1$(Cost)을 이용, Dijkstra로 Shortest Path 계산
* All or Nothing Assignment 실행, y1 획득

In [ ]:
for origin in tqdm(origin_list): # 각 Origin에 대하여 -- Origin은 불변임. OD는 불변이다.
    
    #print(f'origin = {origin}')
    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D']
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination:
            SPD = SPD_Dijkstra(origin, destination, net_cost) ### 변화된 Cost 열을 이용해서 또다시 최단경로 계산(경로가 바뀜)
            link_list = list(SPD['Link_num']) # 바뀐 경로들이 포함한 링크 리스트 나열
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정 
            #print(f'(O, D) = ({origin}, {destination}), Path Link_num list = {link_list}, Add_volume = {add_volume}')
            net_cost.loc[net_cost['Link_num'].isin(link_list), 'y_n'] += add_volume # All or Nothing 배정 :: y_n 갱신
        else:
            pass

In [ ]:
net_cost.head(20)

In [ ]:
sum(net_cost['x_n'])

In [ ]:
sum(net_cost['y_n'])

## Find Move Size $\alpha$
* Finding Direction : $d = y_1-x_1$ 
* 목적함수 $Z(x)$를 최소화하는 $\alpha$ 값 찾기
* 즉, $\sum(y_a^n-x_a^n)*t_a*(x_a^n+\alpha(y_a^n-x_a^n) = 0$이 되는 $\alpha$ 값 찾기

In [ ]:
net_cost['Direction'] = net_cost['y_n'] - net_cost['x_n'] # direction :: d = yn - xn 임

In [ ]:
net_cost['Direction'].unique()

In [ ]:
net_cost

In [ ]:
## Sympy :: 방정식을 풀기 위한 패키지 이용. 방정식에 포함될 각 변수들을 선언하기

d = Symbol('d') # direction
B = Symbol('B') # B. 0.15 있는 그거
t = Symbol('t') # t. Cost
c = Symbol('c') # capacity
x = Symbol('x') # x0
alpha = Symbol('alpha') # alpha
P = Symbol('P') # Power. 4.

In [ ]:
total_equation = 0 # total Equation : Z(x)가 최소화되기 위한 조건식

for i in tqdm(range(1, len(net_cost)+1)): # 1부터 끝까지 Link_num == i에 대하여
    
    d = net_cost.loc[i, 'Direction'] # i링크의 direction
    t = net_cost.loc[i, 'Cost']
    B = net_cost.loc[i, 'B']
    x = net_cost.loc[i, 'x_n']
    P = net_cost.loc[i, 'Power']
    C = net_cost.loc[i, 'Capacity']
    
    equation = d * t * (1 + B * (((x + alpha * d)/C)**P))
    #print(equation)
    
    total_equation += equation

In [ ]:
alpha_list = solve(total_equation) # 여기서 alpha는 0과 1 사이의 양수여야 하므로, 해당 부분만을 필터링해준다.
print(alpha_list)

In [ ]:
def find_alpha(alpha_list):
    """Alpha 값들 중 0 이상 1 이하 실수값만을 반환"""
    for alpha in alpha_list:
        if alpha.is_real == True:
            if alpha <= 1 and alpha >= 0:
                #print(alpha)
                return alpha
            else:
                pass
        else:
            pass

In [ ]:
ith_alpha = find_alpha(alpha_list)
print(ith_alpha) #### 금번 iteration의 alpha를 구했다.

## $x_2$ 계산
* $\alpha$ 값을 이용하여 $x_2$ 값을 계산
* $x_2 = x_1 + \alpha*(y_1 - x_1)$

In [ ]:
net_cost['x_n1'] = net_cost['x_n'] + ith_alpha * (net_cost['Direction'])

In [ ]:
net_cost.head()

## Convergence Test
* 컨버전스 테스트 실행
* 만약 컨버전스 테스트를 통과하지 못할 시, 이 $x_2$를 이용해서 iteration을 반복하게 된다.

In [ ]:
def convergence_test(network):
    """네트워크에 대한 convergence test를 실행하는 함수.
    :: network :: 네트워크 데이터프레임. x_n, x_n1, Cost 열을 포함하고 있어야 함
    :: k :: 수렴 기준. 0.000001 정도로 잡아 본다."""
    sum_xn = sum(network['x_n']) # 분모
    sum_varx = sum((network['x_n1'] - network['x_n']) ** 2)**(1/2) # 분자
    
    criterion = sum_varx / sum_xn
        
    return criterion

In [ ]:
criterion = convergence_test(net_cost)
print(criterion)

In [ ]:
net_cost['x_n'] = net_cost['x_n1']
net_cost['y_n'] = 0 # 초기화해주지 않으면 엄청 늘어나버린다.
net_cost['Direction'] = 0
net_cost['x_n1'] = 0